# Case study 1：添加 Prefetcher

底下是一个很大的脚本，我们重点关心 Shell 函数 `self_build_despacito_stream`

```bash
function self_build_despacito_stream() {
    if [ -f "$gem5_home/../tutorial/xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}
```

在这里，我们创建了一个新的分支 `add_a_new_prefetcher`，并应用了一个 patch `DespacitoStream.patch`。这个 patch 为 GEM5 添加了一个新的预取器 `DespacitoStream`。在应用完 patch 后，我们编译了 GEM5，并将生成的可执行文件保存到 `data/gem5.opt.despacito_stream`。

在进行了一些其他整理操作后，我们运行了 mcf 切片，并计算得到最终的数据。

```bash
function diff_result() {
    pushd $gem5_home > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}
```

In [ ]:
%%bash

#!/usr/bin/env bash

pushd ../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

function run_baseline() {
    gem5_opt_path=$1
    pushd $GEM5_HOME && \
    export LD_LIBRARY_PATH=$GEM5_HOME/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$GEM5_HOME/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_baseline && \
    cd util/xs_scripts/mcf_baseline && \
    $gem5_opt_path $GEM5_HOME/configs/example/xiangshan.py \
    --generic-rv-cpt=$GEM5_HOME/../tutorial/xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$GEM5_HOME/../tutorial/xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function run_with_despacito_stream() {
    gem5_opt_path=$1
    pushd $GEM5_HOME && \
    export LD_LIBRARY_PATH=$GEM5_HOME/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$GEM5_HOME/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_despacito_stream && \
    cd util/xs_scripts/mcf_despacito_stream && \
    $gem5_opt_path $GEM5_HOME/configs/example/xiangshan.py \
    --generic-rv-cpt=$GEM5_HOME/../tutorial/xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$GEM5_HOME/../tutorial/xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function diff_result() {
    pushd $GEM5_HOME > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}

function self_build_baseline() {
    if [ -f "$GEM5_HOME/../tutorial/xs-gem5/data/gem5.opt" ]; then
        return
    fi
    pushd $GEM5_HOME && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt && \
    popd
}

function self_build_despacito_stream() {
    if [ -f "$GEM5_HOME/../tutorial/xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $GEM5_HOME && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}

self_build_baseline && self_build_despacito_stream && \
run_baseline `realpath xs-gem5/data/gem5.opt` && \
run_with_despacito_stream `realpath xs-gem5/data/gem5.opt.despacito_stream` && \
diff_result
